
# Storytime


Before running or if using a different scenario, the user needs to  
make the below changes.  
* Open the scenario 
* Update the file path
* Set the animation time & time step
* In createfolders() decide how the files should be structured  

The goal in this script is to simplify generating eoir synthetic data  
This is done in a few steps 

## _main()_

main() kicks everything off. Using the scenario time, it calculates the  
number of total frames between start and stop. This will be the number  
of times eoirdata() steps through the scenario. The loop jumps to the  
beginning of the scenario then cycles through the following cloud and  
atmospheric parameters.   

|Clouds| . |Atmosphere|
|:-----|:-:|---------:|
|Off   | . |Off       |
|On    | . |Off       |
|On    | . |Simple       |
|On    | . |MODTRAN   |

Once the parameters are set, it dives into eoirdata()

## _eoirdata()_  

eoirdata() uses connect commands to generate the bitmap and the  
raw data. In the same line, the data is stored into it's respective  
folder (see createfolders()). This function then steps forward in time  
and does the whole thing again. The range, or number of iterations is   
set by the number of frames. Once it reaches the end time the function  
jumps out. 

## _createfolders()_  
Create the folders that eoirdata() will save to. It will first check  
if the directories exist and decide if they need to be generated.  

## _Misc_

Replace the manually entered information and let that be entered in as as function

if using custom file, then use  
root.ExecuteCommand('EOIR */ CloudData SetValue MyConstDataSet Altitude 12    
PercentCloudCoverConst 1 PercentCloudCoverFile   
"C:\ProgramData\AGI\STK 12\STKData\VO\Clouds\clouds_2020126.100000_2020127.160000.cld  
EmissivityConst .5 TemperatureConst 233 ForwardScatterConst .1)  


* attach to scenario PID  


# Connect Command Reference

Atmosphere Key
* 0 - off.
* 1 - Simple atmosphere.
* 2 - MODTRAN atmosphere.

Atmosphere Simple Parameters
* Aerosol Models: Rural
* Visibility: 25.00 km
* Humidity: 45.8 %
    
Cloud Key
* On - turns on clouds.
* Off - turns off clouds.

Cloud Parameters
* Altitude: 12 km
* Forward Scatter: 10%
* Back Scatter: 80%
* Constant Coverage: 75%
* Constant Emissivity and Temperature
* Emissivity: 50%
* Temperature: 233K


In [4]:
import os 
import csv
import time
import itertools
import numpy as np
from datetime import timedelta
from datetime import datetime
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# Open EOIR_Clouds_AirToAir.sc
# Connect to the scenario
# Get the IAgStkObjectRoot interface 

stk      = STKDesktop.AttachToApplication()  
root     = stk.Root                           
scenario = root.CurrentScenario 

# Set the scenario animation properties
animation               = scenario.Animation
animation.StartTime     = '5 May 2020 11:41:30.000'
animation.EnableAnimCycleTime = True
animation.AnimCycleTime = '5 May 2020 11:46:30.000'
animation.AnimCycleType = AgEScEndLoopType.eLoopAtTime
animation.AnimStepValue = 0.5


In [5]:
def createfolders(filepath,data,sensor,ftype,cld,atm):

    newdir = os.mkdir("{}output/".format(filepath))

    # Directory structure based on generated data
    for (a,b,c) in itertools.product(data,ftype,sensor): 
        for (x,y) in itertools.zip_longest(cld,atm):
            name = "{}_{}_{}_Cld{}_Atm{}".format(a, b, c, x, y)
            # print("{}output/{}".format(filepath,name))
            os.mkdir("{}output/{}".format(filepath,name))


In [6]:
def eoirdata(filepath,x,y,z):

    # EOIR connect commands SaveSceneImage & SaveSceneRawData 
    command_bmp_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "\
        "\"{}output/raw_bitmap_LWIR".format(filepath))
    command_txt_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "\
        "\"{}output/raw_txt_LWIR".format(filepath))
    command_bmp_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "\
        "\"{}output/raw_bitmap_Visible".format(filepath))
    command_txt_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "\
        "\"{}output/raw_txt_Visible".format(filepath))
    
    # Loop through the entire scenario 
    for i in range(z):
        print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        root.StepForward()


In [7]:
def main():

    # Measure elapsed time for data generation
    start = time.time() 
    
    # Total frames generated based on scenario animation time/timestep
    d1 = datetime.strptime(animation.StartTime,     "%d %b %Y %H:%M:%S.%f")
    d2 = datetime.strptime(animation.AnimCycleTime, "%d %b %Y %H:%M:%S.%f")
    #k  = int(timedelta.total_seconds(d2 - d1)/animation.AnimStepValue)
    k  = 4 # run for testing

    ########
    # USER DEFINED 
    # File path for the output data 
    filepath = 'C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/'

    data    = ['raw','processed'] # the state of the data
    sensor  = ['LWIR','Visible']  # names of the sensors in the scenario 
                                  # potentially remove this and pull in info from scenario
    ftype   = ['bitmap','txt']    # exported data format

    cld     = ['Off','On','On']
    atm     = [0,0,1]
    ########

    # Check if the save to directories exist 
    # Create them before generating data
    isExist = os.path.exists("{}output/".format(filepath))

    if isExist == False:
        print('Creating Directories...')
        createfolders(filepath,data,sensor,ftype,cld,atm)
    else:
        print('Directories Exist')

    # Jump to the beginning of the animation time
    # Set EOIR Cloud and Atmosphere properties
    # Animate and generate data for each timestep 
    for (i,j) in itertools.zip_longest(cld,atm):
        root.Rewind()
        root.ExecuteCommand("EOIR */ SetAtmosphere Clouds {}".format(i))
        root.ExecuteCommand("EOIR */ SetAtmosphere Mode {}".format(j))
        eoirdata(filepath,i,j,k)

    end = time.time()
    print("Elapsed Time (hr): " + str((end - start)/(3600)))

if __name__ == '__main__':
    main()

Directories Exist
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_LWIR_CldOff_Atm0/0.bmp"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_LWIR_CldOff_Atm0/0.txt"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_Visible_CldOff_Atm0/0.bmp"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_Visible_CldOff_Atm0/0.txt"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_LWIR_CldOff_Atm0/1.bmp"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_LWIR_CldOff_Atm0/1.txt"
EOIRDetails */Aircraft/So

In [157]:
sensor  = ['LWIR','Visible'] 
type(sensor)

list

In [120]:
# this counts and iterates over the number of child objects in the scenario
# so now I need to call each individual object and see if it has children
# I should call the objects by type 
# then I can see what's happening from there. 
# I could use item but not sure if I'm doing that right
for i in range(scenario.Children.Count):
    print(i)

0
1
2
3
4
5
6
7
8


In [61]:
scenario.Children.GetElements(AgESTKObjectType.eAircraft).Item(1).HasChildren #this is the miracle # need item to be smarter and then I can use haschildren


True

In [46]:
# for each stk object find item and go to 
# loop with number of children
# to be perfectly safe
# check if count of parent objects children value is greater than 0 yes
# .Item enumerated number 
# .GetElement with the right filter
# 
scParent = scenario.Children
help(scParent.Item) # have to loop through 

Help on method Item in module agi.stk12.stkobjects:

Item(indexOrName:Any) -> 'IAgStkObject' method of agi.stk12.stkobjects.IAgStkObjectCollection instance
    Given an index, returns the element in the collection. If the index is an integer, then method returns the element in the collection at the given position. If the index is a string, then the method returns the element with the specified name.



In [174]:
def findObjects():
    # Ideally this should be able to look through ALL objects in the 
    # scenario and find the child objects... but anyways here we are. 
    parentObj = scenario.Children.GetElements(AgESTKObjectType.eAircraft)

    children = []
    for i in range(parentObj.Count): 
        if parentObj.Item(i).HasChildren == True:
            children.append(i) # capture the identity of the object that has children
        else:
            continue

    children_array = np.array(children)

    sensor = []

    for i in children_array:
        print(parentObj.Item(int(children_array)).ClassName)
        print(parentObj.Item(int(children_array)).InstanceName)
        for j in range(parentObj.Item(int(children_array)).Children.Count):
            print(parentObj.Item(int(children_array)).Children.GetElements(AgESTKObjectType.eSensor).Item(j).ClassName)
            print(parentObj.Item(int(children_array)).Children.GetElements(AgESTKObjectType.eSensor).Item(j).InstanceName)
            sensor.append(parentObj.Item(int(children_array)).Children.GetElements(AgESTKObjectType.eSensor).Item(j).InstanceName)

    print(sensor)

findObjects()

Aircraft
SouthBound
Sensor
LWIR
Sensor
Visible
['LWIR', 'Visible']


In [154]:
scenario.Children.GetElements(AgESTKObjectType.eAircraft).Count # These are Children of the scenario

scenario.Children.GetElements(AgESTKObjectType.eSensor).Count # These are children of the children of the scenario, this isn't looking at the right level

# Loop through all the aircraft objects in the scenario

children = []

for i in range(scenario.Children.GetElements(AgESTKObjectType.eAircraft).Count): #Ideally this should be able to look through ALL objects in the scenario and find the child objects, but anyways here we are. 
    print(i)
    print(scenario.Children.GetElements(AgESTKObjectType.eAircraft).Item(i).HasChildren)
    if scenario.Children.GetElements(AgESTKObjectType.eAircraft).Item(i).HasChildren == True:
        print('I have child objects')
        children.append(i) # capture the identity of the object that has children
    else:
        print('No child Objects')

print(children)

parentObj = scenario.Children.GetElements(AgESTKObjectType.eAircraft)

parentObj.Item(1).Children.Count # correctly identifies the number of children need it to know it's a sensor .... or do I. 
# for robustness I should consider that this needs to sweep for sensors 
parentObj.Item(1).Children.GetElements(AgESTKObjectType.eSensor).Count

childObj = parentObj.Item(1).Children.GetElements(AgESTKObjectType.eSensor)
print(childObj.Item(0).InstanceName,childObj.Item(1).InstanceName)

objPath = "*/{}/{}/{}/{}".format(parentObj.Item(1).ClassName,parentObj.Item(1).InstanceName,childObj.Item(0).ClassName,childObj.Item(0).InstanceName)
print(objPath)
objPath2 = "*/{}/{}/{}/{}".format(parentObj.Item(1).ClassName,parentObj.Item(1).InstanceName,childObj.Item(1).ClassName,childObj.Item(1).InstanceName)
print(objPath2)

import numpy as np
children_array = np.array(children)

for i in children_array:
    print(parentObj.Item(int(children_array)).ClassName)
    print(parentObj.Item(int(children_array)).InstanceName)
    for i in range(parentObj.Item(int(children_array)).Children.Count):
        print(i)
        print(parentObj.Item(int(children_array)).Children.GetElements(AgESTKObjectType.eSensor).Item(i).ClassName)
        print(parentObj.Item(int(children_array)).Children.GetElements(AgESTKObjectType.eSensor).Item(i).InstanceName)


0
False
No child Objects
1
True
I have child objects
2
False
No child Objects
[1]
LWIR Visible
*/Aircraft/SouthBound/Sensor/LWIR
*/Aircraft/SouthBound/Sensor/Visible
Aircraft
SouthBound
0
Sensor
LWIR
1
Sensor
Visible
